# Create KB and dataset

In [1]:
%cd ..

/home/codeholder/code/python-playground/app_noisemon


In [2]:
import json
import os
from pathlib import Path
from tqdm import tqdm
import spacy
from spacy.kb import KnowledgeBase
from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
from wasabi import Printer
msg = Printer()
from scripts.convert_labelstudio_to_spacy import LabelStudioToSpacyConverter

In [3]:
input_path = Path("./data/05-labeled")
output_folder = Path("./corpus/")
kb_path = Path("./corpus/noisemon_kb")

In [4]:
files = list(input_path.glob("*.json"))
if files:
    input_path = max(files, key=os.path.getctime)
else:
    msg.fail(f"Directory {input_path} is empty")

In [5]:
data = json.loads(input_path.read_text())

In [6]:
data[0]

{'id': 334657,
 'annotations': [{'id': 87,
   'completed_by': {'id': 1,
    'email': 'maksim_ermakov@protonmail.com',
    'first_name': '',
    'last_name': ''},
   'result': [{'value': {'start': 6,
      'end': 22,
      'text': 'Открытие Капитал',
      'labels': ['ORG']},
     'id': '74520158391160',
     'from_name': 'ner',
     'to_name': 'text',
     'type': 'labels'},
    {'value': {'start': 6, 'end': 22, 'text': ['Q108398998']},
     'id': '74520158391160',
     'from_name': 'entity',
     'to_name': 'text',
     'type': 'textarea'},
    {'value': {'start': 101,
      'end': 109,
      'text': 'Роснефти',
      'labels': ['ORG']},
     'id': '18591563568703',
     'from_name': 'ner',
     'to_name': 'text',
     'type': 'labels'},
    {'value': {'start': 101, 'end': 109, 'text': ['Q1141123']},
     'id': '18591563568703',
     'from_name': 'entity',
     'to_name': 'text',
     'type': 'textarea'},
    {'value': {'start': 198,
      'end': 206,
      'text': 'Роснефти',
      '

In [17]:
from collections import defaultdict


In [34]:
nlp_model = "ru_core_news_sm"
nlp_model_vector_size = 96
nlp = spacy.load(nlp_model, exclude="parser, tagger")
vocab = nlp.vocab
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=nlp_model_vector_size)

output = []
converter = LabelStudioToSpacyConverter()
converter.ls_label_map = {
    "ORG": "ORG"
}
for labelstudio in tqdm(data):
    id_to_qid_name_pair = defaultdict(dict)
    doc = converter.create_spacy_doc(labelstudio)
    entities = []
    # 1. Matching entities
    for chunk in labelstudio["annotations"][0]["result"]:
        if chunk["from_name"] == "ner":
            id_to_qid_name_pair[chunk["id"]]["text"] = chunk["value"]["text"]
        if chunk["from_name"] == "entity":
            id_to_qid_name_pair[chunk["id"]]["qid"] = chunk["value"]["text"][0]
    # 2. Assigning kb_id to ents
    for chunk in labelstudio["annotations"][0]["result"]:
        if chunk["from_name"] == "ner":
            try:
                entity = doc.char_span(
                    chunk["value"]["start"], 
                    chunk["value"]["end"], 
                    label=chunk["value"]["labels"][0],
                    kb_id=id_to_qid_name_pair[chunk["id"]]["qid"]
                )
                assert entity != None, "Entity failed to be created. Probably misaligned markup"
                entities.append(entity)
            except:
                msg.fail("REsult:", chunk)
                msg.fail("Entity:", None)
                msg.fail("Doc:", doc)
                msg.fail("----------")

    # 3. SEEDING KB ENTITES        
    for _, pair in id_to_qid_name_pair.items():
        if "qid" not in pair:
            msg.fail("QID is missing!", labelstudio["data"]["text"])
            continue
        vector = nlp(labelstudio["data"]["text"]).vector
        kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
        # 4. SEEDING KB ALIASES 
        kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
       
    doc.ents = entities
    output.append(doc)

  0%|                                                                                                    | 0/79 [00:00<?, ?it/s]/tmp/ipykernel_9982/29372325.py:46: UserWarning: [W018] Entity 'Q1141123' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
/tmp/ipykernel_9982/29372325.py:48: UserWarning: [W017] Alias 'Роснефти' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
/tmp/ipykernel_9982/29372325.py:46: UserWarning: [W018] Entity 'Q940518' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
/tmp/ipykernel_9982/29372325.py:48: UserWarning: [W017] Alias 'Магнита' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
/tmp/ipykernel_9982/29372325.py:46: UserWarning: [W018] Entity 'Q1355823' 

 52%|███████████████████████████████████████████████▏                                           | 41/79 [00:02<00:01, 27.18it/s]/tmp/ipykernel_9982/29372325.py:46: UserWarning: [W018] Entity 'Q1884500' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
/tmp/ipykernel_9982/29372325.py:48: UserWarning: [W017] Alias 'ММК' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
/tmp/ipykernel_9982/29372325.py:48: UserWarning: [W017] Alias 'Черкизово' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
/tmp/ipykernel_9982/29372325.py:46: UserWarning: [W018] Entity 'Q171240' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
/tmp/ipykernel_9982/29372325.py:46: UserWarning: [W018] Entity 'Q1368919' alr

✘ REsult:
{'value': {'start': 0, 'end': 4, 'text': 'RUAL', 'labels': ['ORG']}, 'id':
'93478108914436', 'from_name': 'ner', 'to_name': 'text', 'type': 'labels'}
✘ Entity:
✘ Doc:
RUAL отчет Чистая прибыль компании «Русал» за январь—сентябрь 2017 года по ​по
международным стандартам финансовой отчетности (МСФО) ​составила ​$782 млн,
превысив на ​46,4% аналогичный показатель прошлого года. Об этом говорится в
отчете о финансовых результатах, опубликованном на сайте компании. При этом
выручка компании за третий квартал увеличилась на 19,4% и составила $2,46 млрд,
а за девять месяцев текущего года — на 21,3%, до $7,224 млрд. Росту финансовых
показателей способствовало увеличение цены алюминия на Лондонской бирже металлов
(LME), говорится в отчете.
✘ ----------
✘ QID is missing!
RUAL отчет Чистая прибыль компании «Русал» за январь—сентябрь 2017 года по ​по
международным стандартам финансовой отчетности (МСФО) ​составила ​$782 млн,
превысив на ​46,4% аналогичный показатель прошлого года. Об эт

/tmp/ipykernel_9982/29372325.py:46: UserWarning: [W018] Entity 'Q1720713' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
/tmp/ipykernel_9982/29372325.py:48: UserWarning: [W017] Alias 'МегаФона' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
/tmp/ipykernel_9982/29372325.py:48: UserWarning: [W017] Alias 'Алроса' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
/tmp/ipykernel_9982/29372325.py:46: UserWarning: [W018] Entity 'Q1781702' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
/tmp/ipykernel_9982/29372325.py:48: UserWarning: [W017] Alias 'АФК «Система»' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
 80%|███

### At this point we have populated knowledge base `kb`, list of docs with ner and nel markup `output`

In [46]:
kb.get_alias_candidates("CME Group")[0].entity_

'Q1023876'

In [45]:
output[57].ents[0].kb_id_

'Q1840188'

In [14]:
print(f"Entities in the KB: {kb.get_entity_strings()}")
print(f"Aliases in the KB: {kb.get_alias_strings()}")

Entities in the KB: ['Q4218402', 'Q660770', 'Q1141123', 'Q30893504', 'Q193199', 'Q3063197', 'Q4161561', 'Q58707', 'Q2369311', 'Q4044421', 'Q487907', 'Q2632892', 'Q1915579', 'Q1720713', 'Q223799', 'Q841458', 'Q108398998', 'Q2380266', 'Q1477012', 'Q1071853', 'Q251546', 'Q1840188', 'Q1023876', 'Q4047736', 'Q4304175', 'Q108397344', 'Q768773', 'Q171240', 'Q182477', 'Q940518', 'Q2624680', 'Q102673', 'Q173395', 'Q483551', 'Q294508', 'Q329347', 'Q379271', 'Q1368919', 'Q1809133', 'Q1284261', 'Q4258608', 'Q4059809', 'Q1549389', 'Q1461799', 'Q108397243', 'Q1616858', 'Q2304119', 'Q638448', 'Q7907607', 'Q2035424', 'Q2005769', 'Q4038038', 'Q4244736', 'Q1355823', 'Q3656098', 'Q1642605', 'Q108352452', 'Q4513187', 'Q1967957', 'Q4389244', 'Q871308', 'Q1963801', 'Q952937', 'Q130879', 'Q108396966', 'Q108398486', 'Q727452', 'Q2309', 'Q4499024', 'Q4327204', 'Q131723', 'Q1884500', 'Q2116312', 'Q205012', 'Q567050', 'Q1781702', 'Q4102033', 'Q386414', 'Q4400200']
Aliases in the KB: ['Газпром нефти', 'UBS', 'Рус

In [15]:
kb.to_disk(kb_path)

# nlp.to_disk(nlp_dir)

In [49]:
train_docs = DocBin()
dev_docs = DocBin()
test_docs = DocBin()


In [50]:
for i, doc in enumerate(output):
    if i % 10 in (1,3,5):
        dev_docs.add(doc)
    if i % 10 in (4,):
        test_docs.add(doc)
    else:
        train_docs.add(doc)

In [51]:
train_docs.to_disk(output_folder / "train.spacy")
test_docs.to_disk(output_folder / "test.spacy")
dev_docs.to_disk(output_folder / "dev.spacy")